# Senctencepice를 활용한 Vocab 만들기
구글 SentencePice를 활용해 Vocab 만들기. 많은 말뭉치를 사용하는 경우 Vocab을 만드는 것은 어려운 방법

1. character level  
캐릭터 단위로 voacal을 만드는것. 한국어 기준으로, 자음 모음 단어를 기준. 각 단어의 고유한 의미를 표현하지는 않기 때문에 좋은 성능을 내지 못하는 경우가 많다.  
  
2. space level  
띄어쓰기 단위로 vocab. 한국어 경우 조사/ 어미 등으로 인한 중복 단어 문제 발생.

3. subword level  
많은 단어를 처리 가능하고, unknown이 발생할 확률을 줄이는 방법. **단어의 빈도수**를 계산해서 subword 단위로 쪼개는 방법. 단어를 쉽게 쪼갤 수 있도록 google에서 **sentencepiece** 툴 제공. 아래에서는 BPE(Byte Pair Encoding) 사용.


## 1. 말뭉치 만들기(한국어 위키)
vocab을 이용한 말뭉치 필요. 한국어 위키 말뭉치를 사용 예정 [위키백과: 데이터 베이스 다운로드](https://dumps.wikimedia.org/kowiki/20200101/) 에서 1월 1일 자의 page-articles.xml.bz2 파일을 다운로드

web-crawler를 다운 받아서 말뭉치를 만드것까지 한번에 처리
```js
$ git clone https://github.com/paul-hyun/web-crawler.git
$ cd web-crawler
$ pip install tqdm
$ pip install pandas
$ pip install bs4
$ pip install wget
$ pip install pymongo
$ python kowiki.py
```
위 절차를 완료 하면 kowiki 폴더 아래 kowiki_[날짜].csv 형태의 파일 생성

In [5]:
import pandas as pd

in_file = "/Users/a60058238/Desktop/dev/workspace/nlp-study/Pytorch/kowiki/web-crawler/kowiki/kowiki_20200107.csv"
out_file = "/Users/a60058238/Desktop/dev/workspace/nlp-study/Pytorch/kowiki/kowiki.txt"

SEPARATOR = u"\u241D"

df = pd.read_csv(in_file, sep=SEPARATOR, engine ="python")

with open(out_file,"w") as f:
    for index, row in df.iterrows():
        f.write(row["text"]) # 타이틀과 텍스트는 중복되므로 텍스트만 저장
        f.write("\n\n\n\n") # 구분자 

위키데이터의 경우 본문(text)에 제목(title) 정보를 포함하고 있어서 제목과 본문을 둘다 저장할 경우, 내용이 중복된다. 그리고 위키 문서별로 구분하기 위해 줄바꿈을 **4**개로 주었다

> 아나콘다 설치 시 오류


In [16]:
import Pytorch.sentencepiece as spm

spm.SetencePieceTrainer()

ModuleNotFoundError: No module named 'Pytorch'

## 2. Google Sentencepiece 설치
Anaconda & Mac 환경에서 setencepiece를 설치하는데 어려움.
```
conda install -c roccqqck sentencepiece
``` 
으로 sentencepiece 패키지 설치

## 3. Vocab 만들기

In [19]:
import sentencepiece as spm

corpus = "kowiki.txt"
prefix = "kowiki"

vocab_size = 8000

spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

AttributeError: module 'sentencepiece' has no attribute 'SentencePieceTrainer'

**SentencePieceTrainer.train** 옵션
- input: 입력 corpus
- prefix: 저장할 모델 이름
- vocab_size: vocab 개수 (기본 8,000에 스페셜 토큰 7개를 더해서 8,007개)
- max_sentence_length: 문장의 최대 길이
- pad_id, pad_piece: pad token id, 값
- unk_id, unk_piece: unknown token id, 값
- bos_id, bos_piece: begin of sentence token id, 값
- eos_id, eos_piece: end of sequence token id, 값
- user_defined_symbols: 사용자 정의 토큰

> vocab_size의 경우 Etri KoBERT의 경우 32,000개 sktKoBERT의 경우 8,000개 사용.
> vocab_size가 커지면 성능이 좋아지고, 모델 파라미터 수는 증가


vocab 생성이 완료되면, kowiki.model과 kowiki.vocab 파일이 생성된다

## 4. Vocab 테스트
생성된 vocab을 이용한 간단한 테스트 코드

In [21]:
import sys
sys.path

['/Users/a60058238/Desktop/dev/workspace/nlp-study/Pytorch',
 '/Users/a60058238/opt/anaconda3/lib/python37.zip',
 '/Users/a60058238/opt/anaconda3/lib/python3.7',
 '/Users/a60058238/opt/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Users/a60058238/.local/lib/python3.7/site-packages',
 '/Users/a60058238/opt/anaconda3/lib/python3.7/site-packages',
 '/Users/a60058238/opt/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Users/a60058238/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/a60058238/.ipython']

In [20]:
import sentencepiece as spm

In [22]:
vocab_file = "<path of vocab>/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  print(line)
  print(pieces)
  print(ids)
  print()

AttributeError: module 'sentencepiece' has no attribute 'SentencePieceProcessor'

위 프로그램의 subword 단위로 쪼개는 것을 확인할 수 있다.

쪼개진 결과는 아래와 같다
```
겨울이 되어서 날씨가 무척 추워요.
['▁겨울', '이', '▁되어', '서', '▁날', '씨', '가', '▁무', '척', '▁추', '워', '요', '.']
[3091, 3588, 601, 3602, 683, 4014, 3599, 108, 4193, 206, 3958, 3760, 3590]

이번 성탄절은 화이트 크리스마스가 될까요?
['▁이번', '▁성', '탄', '절', '은', '▁화', '이트', '▁크리스', '마', '스가', '▁될', '까', '요', '?']
[3224, 86, 3967, 3923, 3604, 264, 669, 1970, 3664, 780, 1450, 3794, 3760, 4245]

겨울에 감기 조심하시고 행복한 연말 되세요.
['▁겨울', '에', '▁감', '기', '▁조', '심', '하', '시', '고', '▁행', '복', '한', '▁연', '말', '▁되', '세', '요', '.']
[3091, 3591, 212, 3605, 53, 3832, 3596, 3613, 3600, 234, 3871, 3603, 61, 3823, 445, 3682, 3760, 3590]
```


# References
https://paul-hyun.github.io/vocab-with-sentencepiece/